In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# Load the preprocessed CSV file
df = pd.read_csv('../data/lemmatized_and_misspelled_removed_SEFACED.csv')

# Split the data into features (lemmatized text) and target variable
X = df['lemmatized_tokens']
y = df['Class_Label']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Tokenize the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_seq = tokenizer.texts_to_sequences(X)

# Define maximum sequence length
maxlen = 100

# Pad sequences to ensure uniform length
X_pad = pad_sequences(X_seq, maxlen=maxlen)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_pad, y_encoded, test_size=0.2, random_state=42)

# One-hot encode the labels
num_classes = len(np.unique(y_train))
y_train_one_hot = np.eye(num_classes)[y_train]
y_test_one_hot = np.eye(num_classes)[y_test]

# Define LSTM model architecture
embedding_dim = 100

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, input_length=maxlen))
model.add(LSTM(128))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))  

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_test, y_test_one_hot))

# Evaluate the model
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)

print(classification_report(y_test, y_pred))


Epoch 1/10
400/400 [==============================] - 95s 222ms/step - loss: 0.5848 - accuracy: 0.7646 - val_loss: 0.3096 - val_accuracy: 0.8703
Epoch 2/10
400/400 [==============================] - 95s 237ms/step - loss: 0.2391 - accuracy: 0.9116 - val_loss: 0.2924 - val_accuracy: 0.8813
Epoch 3/10
400/400 [==============================] - 78s 195ms/step - loss: 0.1730 - accuracy: 0.9364 - val_loss: 0.3282 - val_accuracy: 0.8784
Epoch 4/10
400/400 [==============================] - 78s 196ms/step - loss: 0.1392 - accuracy: 0.9463 - val_loss: 0.3701 - val_accuracy: 0.8772
Epoch 5/10
400/400 [==============================] - 81s 202ms/step - loss: 0.1303 - accuracy: 0.9517 - val_loss: 0.4297 - val_accuracy: 0.8778
Epoch 6/10
400/400 [==============================] - 79s 198ms/step - loss: 0.1169 - accuracy: 0.9539 - val_loss: 0.4368 - val_accuracy: 0.8753
Epoch 7/10
400/400 [==============================] - 79s 198ms/step - loss: 0.0979 - accuracy: 0.9610 - val_loss: 0.4710 - val_ac